In [2]:
%pip install numpy
%pip install matplotlib
%pip install cv2
%pip install sklearn
%pip install pandas
%pip install tensorflow

^C
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
import os
import re
from sklearn.model_selection import train_test_split
import pandas as pd

ModuleNotFoundError: No module named 'numpy'

In [ ]:
%git clone https://github.com/christopherevan/CV_Shoe_Dataset.git

In [ ]:
%cd CV_Shoe_Dataset
%unzip test.zip
%unzip train.zip
%unzip valid.zip

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train_data_generator = train_datagen.flow_from_directory("Train", 
                                                      target_size=(224, 224), batch_size=8, 
                                                      class_mode = "categorical", shuffle = True)
test_data_generator = test_datagen.flow_from_directory("Test", 
                                                      target_size=(224, 224), batch_size=8, 
                                                      class_mode = "categorical", shuffle = True)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

data_augmentation = keras.Sequential(
    [layers.RandomFlip("horizontal"), layers.RandomRotation(0.1),]
)

In [ ]:
batch = next(train_data_generator)
plt.figure(figsize=(10, 10))
first_image = batch[0][0]
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    augmented_image = data_augmentation(
        tf.expand_dims(first_image, 0), training=True
    )
    plt.imshow(augmented_image[0].numpy().astype("int32"))
    plt.axis("off")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras import models, layers, optimizers

In [ ]:
from tensorflow.keras.applications import EfficientNetB0

model_base = EfficientNetB0(
            include_top=False,
            weights="imagenet")    

model_base.trainable = False

inputs = keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)

x = model_base(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)
predictions = layers.Dense(int(len(train_data_generator.class_indices.keys())), activation='softmax')(x)
model = models.Model(inputs=inputs, outputs=predictions)
model.summary()


In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy', tf.keras.metrics.TruePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives()])
hist = model.fit(train_data_generator,
                      epochs = 25,
                      validation_data = test_data_generator)

In [ ]:
model_base.trainable = True
model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss = 'categorical_crossentropy', metrics = ['accuracy', tf.keras.metrics.TruePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives()]
)

epochs = 15
hist = model.fit(train_data_generator, epochs=epochs, validation_data=test_data_generator)

In [ ]:
from tensorflow.keras.models import save_model
save_model(model, "eff_b0_shoe.h5")